### File paths

In [22]:
import pandas as pd
import re

synonym_file = "combined_synonym_list.xlsx"
pauls_list_file = "(Paul-list task)Plant_Species_Native_Exotic_Genus.xlsx"
pct_lists_file = "CBCity PCT Lists.xlsx"
occurrence_file = "occurrences_new.csv"
output_file = "Unrecorded_Native_Species_PCT.xlsx"

### Build synonym dictionary

In [23]:
# Build synonym mapping dictionary
syn_df = pd.read_excel(synonym_file)
syn_df.columns = syn_df.columns.str.strip().str.lower()

# Build alternate → accepted mapping
syn_dict = dict(zip(
    syn_df['alternate_names'].astype(str).str.strip(),
    syn_df['accepted_name'].astype(str).str.strip()
))

def apply_synonym(name: str):
    """Map alternate species name → accepted species name"""
    if pd.isna(name):
        return None
    name = str(name).strip()
    return syn_dict.get(name, name)

print(f"Synonym dictionary loaded ({len(syn_dict)} mappings)")

# Paul's List (Native_list only, no synonym handling need) 
paul_native = pd.read_excel(pauls_list_file, sheet_name="Native_list")
before_unique = paul_native['scientific_name'].nunique()
print(f"Paul's List Native_list loaded ({before_unique} unique species, no synonym mapping applied)")

# CBCity PCT Lists (all sheets need synonym handling) 
cbc_xls = pd.ExcelFile(pct_lists_file)
cbc_sheets = {}

for sheet in cbc_xls.sheet_names:
    # Load with flexible typing
    df = pd.read_excel(pct_lists_file, sheet_name=sheet, dtype=str)
    df.columns = df.columns.str.strip()

    # Identify the 'species' column 
    sp_col = [c for c in df.columns if 'species' in c.lower()]
    if not sp_col:
        print(f"Sheet '{sheet}' skipped: no 'species' column found.")
        continue

    df.rename(columns={sp_col[0]: 'Species'}, inplace=True)

    # Apply synonym mapping
    df['Original_Name'] = df['Species']
    df['Species'] = df['Species'].apply(apply_synonym)

    # Remove empty or group-header rows
    df = df[~df['Species'].isna()]
    df = df[~df['Species'].astype(str).str.startswith(':')]

    header_regex = r'^(tree|trees|shrub|shrubs|herb|herbs|grass|grasses|sedge|sedges|vine|vines|fern|ferns|understorey|understory|ground\s*cover|forb|forbs|graminoid|rush|rushes)$'
    df = df[~df['Species'].astype(str).str.strip().str.lower().str.match(header_regex, na=False)]

    # Clean text fields
    df['Species'] = df['Species'].astype(str).str.strip()
    if 'Growth Form' in df.columns:
        df['Growth Form'] = df['Growth Form'].astype(str).str.strip()

    # Store cleaned sheet
    cbc_sheets[sheet] = df

print(f"CBCity PCT Lists synonym mapping done ({len(cbc_sheets)} sheets processed)")

# Occurrence table (no synonym handling need)
occ_df = pd.read_csv(occurrence_file, dtype=str, low_memory=False)
before_occ = occ_df['scientific_name'].nunique()
print(f"Occurrence table loaded ({before_occ} unique species, no synonym mapping applied)")

# Summary
print("Synonym mapping completed successfully:")
print(f"Paul's List → Native_list ({len(paul_native)} rows)")
print(f"CBCity PCT Lists → {len(cbc_sheets)} sheets")
print(f"Occurrence table → {len(occ_df)} rows\n")

Synonym dictionary loaded (11939 mappings)
Paul's List Native_list loaded (1201 unique species, no synonym mapping applied)
CBCity PCT Lists synonym mapping done (26 sheets processed)
Occurrence table loaded (2103 unique species, no synonym mapping applied)
Synonym mapping completed successfully:
Paul's List → Native_list (1201 rows)
CBCity PCT Lists → 26 sheets
Occurrence table → 33439 rows



### Identify recorded native species from Occurrence table

In [24]:
# Extract the collection of native species from Paul's List
native_species = set(paul_native['scientific_name'].dropna().unique())
print(f"Native species loaded from Paul's List ({len(native_species)} species)")

# Ensure that the necessary columns of the occurrence table exist
required_cols = {'scientific_name', 'pct_id'}
if not required_cols.issubset(occ_df.columns):
    missing = required_cols - set(occ_df.columns)
    raise KeyError(f"Missing required columns in occurrence table: {missing}")

# Filter the records belonging to native species
occ_native = occ_df[occ_df['scientific_name'].isin(native_species)].copy()

# Keep the two required columns and clear the null values
occ_native = occ_native[['scientific_name', 'pct_id']].dropna(subset=['pct_id'])
occ_native['pct_id'] = occ_native['pct_id'].astype(str).str.strip()

# If the pct_id on the same line contains multiple codes (separated by commas), expand it into multiple lines
occ_native = occ_native.assign(pct_id=occ_native['pct_id'].str.split(',')).explode('pct_id')
occ_native['pct_id'] = occ_native['pct_id'].str.strip()
occ_native = occ_native[occ_native['pct_id'].str.lower() != 'none']

# Deduplication (A species-PCT combination is retained only once)
occ_native = occ_native.drop_duplicates(subset=['scientific_name', 'pct_id']).reset_index(drop=True)

# Merge genus & family from Paul's List
if not {'scientific_name', 'genus', 'family_name'}.issubset(paul_native.columns):
    raise KeyError("Paul's List must include 'scientific_name', 'genus', and 'family_name' columns.")

# Perform a left merge to attach genus and family info
# Deduplicate to prevent expansion
paul_native_unique = paul_native.drop_duplicates(subset=['scientific_name']).copy()
occ_native = occ_native.merge(
    paul_native_unique[['scientific_name', 'genus', 'family_name']],
    on='scientific_name',
    how='left'
)

# Recorded species list ——> Group by PCT
recorded_output = "recorded_native_species_PCT.xlsx"
with pd.ExcelWriter(recorded_output, engine='openpyxl') as writer:
    for pct in sorted(occ_native['pct_id'].dropna().unique()):
        subset = occ_native[occ_native['pct_id'] == pct][['scientific_name', 'genus', 'family_name', 'pct_id']]
        sheet_name = str(pct)[:31]
        subset.to_excel(writer, index=False, sheet_name=sheet_name)
print(f"Recorded native species written to: {recorded_output}")

Native species loaded from Paul's List (1201 species)
Recorded native species written to: recorded_native_species_PCT.xlsx


### Extract Expected Native List (per PCT)

In [25]:
expected_native_dict = {}

# Obtain the native species set from Paul's List
native_species = set(paul_native['scientific_name'].dropna().unique())
print(f"Loaded {len(native_species)} native species from Paul's List")

# Traverse all sheets of CBCity PCT Lists
for sheet_name, df in cbc_sheets.items():
    df.columns = df.columns.str.strip()
    
    # Dynamically identify the "species" column
    sp_col = [c for c in df.columns if 'species' in c.lower()]
    if not sp_col:
        print(f"Sheet '{sheet_name}' skipped: no species column found.")
        continue

    df.rename(columns={sp_col[0]: 'Species'}, inplace=True)
    
    # Step 1: Clear blank lines and title lines
    df = df[~df['Species'].isna()]                         
    df = df[~df['Species'].astype(str).str.startswith(':')] 
    
    # Step 2: Clean the string
    df['Species'] = df['Species'].astype(str).str.strip()
    
    # Step 3: Filter native species
    native_df = df[df['Species'].isin(native_species)].copy()
    
    # Step 4: Only keep the columns of concern
    cols_to_keep = ['Original_Name', 'Species', 'Growth Form', 'Median Cover Score', 'Frequency']
    existing_cols = [c for c in cols_to_keep if c in native_df.columns]
    native_df = native_df[existing_cols]
    
    # Step 5: Extract numeric PCT code
    pct_match = re.search(r'(\d+)', sheet_name)
    pct_code = pct_match.group(1) if pct_match else str(sheet_name)
    native_df['PCT_code'] = pct_code
 
    # Step 6: Remove duplicates
    native_df = native_df.drop_duplicates(subset=['Species']).reset_index(drop=True)
    
    # Store using numeric PCT code as key
    expected_native_dict[pct_code] = native_df
    print(f"{pct_code}: {len(native_df)} native species extracted.")

# ---------- Output the results ----------
with pd.ExcelWriter("expected_native_species_PCT.xlsx", engine='openpyxl') as writer:
    for sheet_name, subdf in expected_native_dict.items():
        # Extract the pure numeric PCT code from the worksheet name
        match = re.search(r'\d+', str(sheet_name))
        export_name = match.group(0) if match else str(sheet_name)
        subdf.to_excel(writer, index=False, sheet_name=export_name[:31])
        print(f"Sheet '{export_name}' written ({len(subdf)} records)")

print("Saved as: expected_native_species_PCT.xlsx")

Loaded 1201 native species from Paul's List
3040: 192 native species extracted.
3136: 227 native species extracted.
3259: 317 native species extracted.
3262: 350 native species extracted.
3320: 348 native species extracted.
3448: 339 native species extracted.
3592: 399 native species extracted.
3594: 257 native species extracted.
3595: 393 native species extracted.
3615: 318 native species extracted.
3619: 369 native species extracted.
3628: 104 native species extracted.
3629: 236 native species extracted.
3813: 244 native species extracted.
3814: 250 native species extracted.
3963: 37 native species extracted.
3972: 31 native species extracted.
3985: 63 native species extracted.
4006: 200 native species extracted.
4023: 148 native species extracted.
4024: 122 native species extracted.
4027: 84 native species extracted.
4028: 124 native species extracted.
4057: 155 native species extracted.
4091: 21 native species extracted.
4097: 33 native species extracted.
Sheet '3040' written (192 

### Unrecorded native species detection

In [26]:
# File paths
expected_file = "expected_native_species_PCT.xlsx"
recorded_file = "recorded_native_species_PCT.xlsx"
output_file = "unrecorded_native_species_PCT.xlsx"

# Load workbooks
expected_xls = pd.ExcelFile(expected_file)
recorded_xls = pd.ExcelFile(recorded_file)

expected_sheets = expected_xls.sheet_names
recorded_sheets = recorded_xls.sheet_names

# Build intersection of PCT sheet names
common_sheets = sorted(set(expected_sheets) & set(recorded_sheets))
print(f"Found {len(common_sheets)} common PCT sheets to compare")

unrecorded_results = {}

# Compare each PCT
for sheet in common_sheets:
    # Read expected native species
    exp_df = pd.read_excel(expected_xls, sheet_name=sheet, dtype=str)
    exp_df.columns = exp_df.columns.str.strip()
    if "Species" not in exp_df.columns:
        print(f"Sheet '{sheet}' skipped: missing 'Species' column")
        continue

    # Read recorded native species
    rec_df = pd.read_excel(recorded_xls, sheet_name=sheet, dtype=str)
    rec_df.columns = rec_df.columns.str.strip()
    if "scientific_name" not in rec_df.columns:
        continue

    # Clean and normalize names
    exp_df["Species"] = exp_df["Species"].astype(str).str.strip()
    rec_df["scientific_name"] = rec_df["scientific_name"].astype(str).str.strip()

    # Compute difference set
    exp_species = set(exp_df["Species"].dropna().unique())
    rec_species = set(rec_df["scientific_name"].dropna().unique())
    unrecorded_species = exp_species - rec_species
    
    # Filter expected data for unrecorded species
    unrec_df = exp_df[exp_df["Species"].isin(unrecorded_species)].copy()
    
    unrec_df.rename(columns={"Species": "scientific_name"}, inplace=True)
    unrec_df = unrec_df.merge(
        paul_native_unique[["scientific_name", "genus", "family_name"]],
        on="scientific_name",
        how="left"
    )
    
    # Reorder columns
    final_cols = [
        "scientific_name", "genus", "family_name",
        "Growth Form", "Median Cover Score", "Frequency"
    ]
    unrec_df = unrec_df[[c for c in final_cols if c in unrec_df.columns]]
    
    # Save result
    unrecorded_results[sheet] = unrec_df
    print(f"{sheet}: {len(unrec_df)} unrecorded species")

# Save all results (Refine the display as requested)
final_output_file = "unrecorded_native_species_PCT.xlsx"
with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for sheet_name, subdf in unrecorded_results.items():
        subdf.to_excel(writer, index=False, sheet_name=str(sheet_name)[:31])

print(f"Unrecorded native species file saved as: {output_file} ")

Found 26 common PCT sheets to compare
3040: 169 unrecorded species
3136: 221 unrecorded species
3259: 163 unrecorded species
3262: 103 unrecorded species
3320: 106 unrecorded species
3448: 83 unrecorded species
3592: 222 unrecorded species
3594: 131 unrecorded species
3595: 391 unrecorded species
3615: 120 unrecorded species
3619: 139 unrecorded species
3628: 51 unrecorded species
3629: 195 unrecorded species
3813: 120 unrecorded species
3814: 221 unrecorded species
3963: 28 unrecorded species
3972: 30 unrecorded species
3985: 43 unrecorded species
4006: 199 unrecorded species
4023: 136 unrecorded species
4024: 12 unrecorded species
4027: 83 unrecorded species
4028: 32 unrecorded species
4057: 110 unrecorded species
4091: 4 unrecorded species
4097: 27 unrecorded species
Unrecorded native species file saved as: unrecorded_native_species_PCT.xlsx 
